In [2]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

memory.save_context({"input":"Hi!"},{"output":"How are you?"})

memory.load_memory_variables({})

{'history': 'Human: Hi!\nAI: How are you?'}

In [4]:
memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input":"Hi!"},{"output":"How are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!'), AIMessage(content='How are you?')]}

In [5]:
memory.save_context({"input":"Hi!"},{"output":"How are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!'),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi!'),
  AIMessage(content='How are you?')]}